In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import numpy as np

'1.5.0'

In [2]:
OUTPUT_DIR = "./models"
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

tokenizer = GPT2Tokenizer.from_pretrained(OUTPUT_DIR)
model = GPT2LMHeadModel.from_pretrained(OUTPUT_DIR)
model = model.to(device)

In [3]:
def generate(input_str, length=250, n=5):
    cur_ids = torch.tensor(tokenizer.encode(input_str)).unsqueeze(0).long().to(device)
    model.eval()
    with torch.no_grad():
        for i in range(length):
            outputs = model(cur_ids[:, -1024:], labels=cur_ids[:, -1024:])
            loss, logits = outputs[:2]
            softmax_logits = torch.softmax(logits[0,-1], dim=0)
            next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n)
            cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim=1)
        output_list = list(cur_ids.squeeze().to('cpu').numpy())
        output_text = tokenizer.decode(output_list)
        return output_text

def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

In [7]:
# generated_text = generate("This aparatus \n", n=5)
generated_text = generate("<BOS>", n=5)
print(generated_text)

<BOS> 1.  A display panel comprising: a plurality of pixel cells arranged in a first direction, each pixel cell comprising a first pixel region, a second pixel region, and a fourth pixel region;  an input layer disposed on the first pixel region, the input layer including: at least one first pixel region, a second pixel region, and a third pixel region on the first pixel region and the second pixel region, the first pixel region and the second pixel region each being arranged along a first direction parallel to a first edge of the pixel region;  the second pixel region on the first pixel region and on the second pixel region, the second pixel region and the third pixel region each being arranged along a second direction parallel to a second edge of the pixel region;  a first pixel region disposed between a third pixel region and a fourth pixel region;  a third pixel region disposed between a fourth pixel region and an upper surface of the pixel region;  a light emitting device on each 